<a href="https://colab.research.google.com/github/MacherelR/AutonomousRcCar/blob/main/ModelQuantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
model_output_dir = '/content/gdrive/My Drive/Colab Notebooks/LaneNavigation'

import sys
sys.path.insert(0,'/content/gdrive/MyDrive/Colab Notebooks')

Mounted at /content/gdrive


In [2]:
# python standard libraries
import os
import random
import fnmatch
import datetime
import pickle

# data processing
import numpy as np
np.set_printoptions(formatter={'float_kind':lambda x: "%.4f" % x})

import pandas as pd
pd.set_option('display.width', 300)
pd.set_option('display.float_format', '{:,.4f}'.format)
pd.set_option('display.max_colwidth', 200)

# tensorflow
import tensorflow as tf
import keras
from keras.models import Sequential  # V2 is tensorflow.keras.xxxx, V1 is keras.xxx
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.models import load_model

print( f'tf.__version__: {tf.__version__}' )
print( f'keras.__version__: {keras.__version__}' )

# sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# imaging
import cv2
from imgaug import augmenters as img_aug
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from PIL import Image



import csv

import ML_Lib

tf.__version__: 2.5.0
keras.__version__: 2.5.0


In [3]:
!cd /content
if not os.path.isdir("AutonomousRcCar"):
  !git clone https://github.com/MacherelR/AutonomousRcCar
!ls

dataDirectory = '/content/AutonomousRcCar/Datas/DataCollection' # DataCollection
steeringList,img_path,namesList = ML_Lib.readFiles(dataDirectory)
df = pd.DataFrame()
df['ImagePath'] = img_path
df['Angle'] = steeringList


Cloning into 'AutonomousRcCar'...
remote: Enumerating objects: 7383, done.
remote: Counting objects: 100% (272/272), done.
remote: Compressing objects: 100% (269/269), done.
remote: Total 7383 (delta 4), reused 269 (delta 3), pack-reused 7111
Receiving objects: 100% (7383/7383), 775.61 MiB | 31.02 MiB/s, done.
Resolving deltas: 100% (285/285), done.
Checking out files: 100% (7973/7973), done.
AutonomousRcCar  gdrive  sample_data


In [4]:
 model = load_model(f'{model_output_dir}/lane_navigation_TestlrDown.h5')

**Quantification de la plage dynamique**

In [ ]:
newDir = '/content/gdrive/My Drive/Colab Notebooks/Models/'
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Save the model.
with open(f'{newDir}/modelConvertedQT3.tflite', 'wb') as f:
  f.write(tflite_model)

**Quantification entier-entier**

In [11]:
def representativeGen():
  for i in range(150):
    randomIdx = random.randint(0,len(img_path)-1)
    imgPath = img_path[randomIdx]
    image = ML_Lib.imreadModif(imgPath)
    steer = steeringList[randomIdx]
    preprocessed = ML_Lib.ImagePreprocess(image)
    input_data = np.expand_dims(np.float32(preprocessed), axis=0)
    yield [input_data]

In [14]:
newDir = '/content/gdrive/My Drive/Colab Notebooks/QuantizationIntInt'

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.representative_dataset = representativeGen

tflite_model = converter.convert()

# Save the model.
with open(f'{newDir}/modelConvertedQT.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmph_a9dvpc/assets


INFO:tensorflow:Assets written to: /tmp/tmph_a9dvpc/assets


**Comparaison de tailles**

In [17]:
DynIntModelPath = '/content/gdrive/My Drive/Colab Notebooks/Models/modelConvertedQT2lrDown.tflite'
IntIntModelPath = f'{newDir}/modelConvertedQT.tflite'

print("Dynamically quantized model in Mb:", os.path.getsize(DynIntModelPath) / float(2**20))
print("Int Int Quantized model in Mb:", os.path.getsize(IntIntModelPath) / float(2**20))
print("Compression ratio:", os.path.getsize(DynIntModelPath)/os.path.getsize(IntIntModelPath))

Dynamically quantized model in Mb: 0.25421142578125
Int Int Quantized model in Mb: 0.25609588623046875
Compression ratio: 0.9926415825066285
